# Detecting Data Drift in NLP using SageMaker Custom Model Monitor

This Example is an extension of [Fine-tuning a PyTorch BERT model and deploying it with Amazon Elastic Inference on Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/fine-tuning-a-pytorch-bert-model-and-deploying-it-with-amazon-elastic-inference-on-amazon-sagemaker/) aws blog post. We will use the dataset and model thats outlined in this blog and extend it to demo custom model monitoring capability using [SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)

Detecting data drift in NLP is a challenging task. Model monitoring becomes an important aspect in MLOPs because the change in data distribution during inference time can cause Model decay. ML models are probabilistic and trained on certain corpus of historical data. Drift is distribution change between the training and deployment data, which is concerning if model performance changes.

We will begin with creating PyTorch Model using previously trained model artifacts. We will deploy the model to a SageMaker real time endpoint. To establish a baseline of training data distribution we will caluclate BERT sentence embedding and use that in the custom model monitoring scripts to compare the real time inferece traffic to compare a distance metrics to determine the deviation fron training distribution

![title](images/nlp-data-drift-bert.png)

# Setup

To start, we import some Python libraries and initialize a SageMaker session, S3 bucket and prefix, and IAM role.

In [93]:
# need torch 1.3.1 for elastic inference
!python -m pip install --upgrade pip
!pip install torch==1.3.1 --quiet
!pip install transformers --quiet

In [95]:
import os
import numpy as np
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
model_prefix = "sagemaker/nlp-data-drift-bert-model"

role = sagemaker.get_execution_role()

# Prepare training data

We use Corpus of Linguistic Acceptability (CoLA) (https://nyu-mll.github.io/CoLA/), a dataset of 10,657 English sentences labeled as grammatical or ungrammatical from published linguistics literature. We download and unzip the data using the following code:

### Download data

In [96]:
if not os.path.exists("./cola_public_1.1.zip"):
    !curl -o ./cola_public_1.1.zip https://nyu-mll.github.io/CoLA/cola_public_1.1.zip
if not os.path.exists("./cola_public/"):
    !unzip cola_public_1.1.zip

### Get sentences and labels

Let us take a quick look at our data. First we read in the training data. The only two columns we need are the sentence itself and its label. 

In [97]:
df = pd.read_csv(
    "./cola_public/raw/in_domain_train.tsv",
    sep="\t",
    header=None,
    usecols=[1, 3],
    names=["label", "sentence"],
)
sentences = df.sentence.values
labels = df.label.values

In [98]:
df.head()

,label,sentence
0,1,"Our friends won't buy this analysis, let alone..."
1,1,One more pseudo generalization and I'm giving up.
2,1,One more pseudo generalization or I'm giving up.
3,1,"The more we study verbs, the crazier they get."
4,1,Day by day the facts are getting murkier.


Printing out a few sentences shows us how sentences are labeled based on their grammatical completeness. 

In [99]:
print(sentences[20:25])
print(labels[20:25])

['The professor talked us.' 'We yelled ourselves hoarse.'
 'We yelled ourselves.' 'We yelled Harry hoarse.'
 'Harry coughed himself into a fit.']
[0 1 0 0 1]


We then split the dataset for training and testing.

In [100]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df)
train.to_csv("./cola_public/train.csv", index=False)
test.to_csv("./cola_public/test.csv", index=False)

Next, we upload both to Amazon S3 for use later. The SageMaker Python SDK provides a helpful function for uploading to Amazon S3:

In [101]:
inputs_train = sagemaker_session.upload_data("./cola_public/train.csv", bucket=bucket, key_prefix=model_prefix)
inputs_test = sagemaker_session.upload_data("./cola_public/test.csv", bucket=bucket, key_prefix=model_prefix)

# Run training

## Training script

We use the [PyTorch-Transformers library](https://pytorch.org/hub/huggingface_pytorch-transformers), which contains PyTorch implementations and pre-trained model weights for many NLP models, including BERT.

Our training script should save model artifacts learned during training to a file path called `model_dir`, as stipulated by the SageMaker PyTorch image. Upon completion of training, model artifacts saved in `model_dir` will be uploaded to S3 by SageMaker and will become available in S3 for deployment.

We save this script in a file named `train_deploy.py`, and put the file in a directory named `code/`. The full training script can be viewed under `code/`.

In [102]:
!pygmentize code/train_deploy.py

import argparse
import json
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
import transformers
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, BertForSequenceClassification, BertTokenizer

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 64  # this is the max length of the sentence

print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def _get_train_data_loader(batch_size, training_dir, is_distributed):
    logger.info("Get train data loader")

    dataset = pd.read_csv(o

## Train on Amazon SageMaker

We use Amazon SageMaker to train and deploy a model using our custom PyTorch code. The Amazon SageMaker Python SDK makes it easier to run a PyTorch script in Amazon SageMaker using its PyTorch estimator. After that, we can use the SageMaker Python SDK to deploy the trained model and run predictions. For more information on how to use this SDK with PyTorch, see [the SageMaker Python SDK documentation](https://sagemaker.readthedocs.io/en/stable/using_pytorch.html).

To start, we use the `PyTorch` estimator class to train our model. When creating our estimator, we make sure to specify a few things:

* `entry_point`: the name of our PyTorch script. It contains our training script, which loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model. It also contains code to load and run the model during inference.
* `source_dir`: the location of our training scripts and requirements.txt file. "requirements.txt" lists packages you want to use with your script.
* `framework_version`: the PyTorch version we want to use

The PyTorch estimator supports multi-machine, distributed PyTorch training. To use this, we just set train_instance_count to be greater than one. Our training script supports distributed training for only GPU instances. 

After creating the estimator, we then call fit(), which launches a training job. We use the Amazon S3 URIs where we uploaded the training data earlier.

In [103]:
from sagemaker.pytorch import PyTorch

# place to save model artifact
output_path = f"s3://{bucket}/{model_prefix}"

estimator = PyTorch(
    entry_point="train_deploy.py",
    source_dir="code",
    role=role,
    framework_version="1.3.1",
    py_version="py3",
    instance_count=1,  # this script only support distributed training for GPU instances.
    instance_type="ml.p3.2xlarge",
    output_path=output_path,
    hyperparameters={
        "epochs": 1,
        "num_labels": 2,
        "backend": "gloo",
    },
    disable_profiler=True, # disable debugger
)
estimator.fit({"training": inputs_train, "testing": inputs_test})

2021-08-12 21:14:46 Starting - Starting the training job...
2021-08-12 21:14:49 Starting - Launching requested ML instances......
2021-08-12 21:16:04 Starting - Preparing the instances for training............
2021-08-12 21:17:53 Downloading - Downloading input data...
2021-08-12 21:18:26 Training - Downloading the training image......
2021-08-12 21:19:45 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-12 21:19:46,535 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-08-12 21:19:46,561 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-12 21:19:49,576 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-12 21:19:49,888 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
202

Loading BERT tokenizer...
INFO:__main__:Train Epoch: 1 [0/6413 (0%)] Loss: 0.632982
Distributed training - False
INFO:__main__:Train Epoch: 1 [3200/6413 (50%)] Loss: 0.448342
Number of gpus available - 1
INFO:__main__:Train Epoch: 1 [1300/6413 (99%)] Loss: 0.205874
Get train data loader
INFO:__main__:Average training loss: 0.509086
Processes 6413/6413 (100%) of train data

Processes 2138/2138 (100%) of test data
INFO:__main__:Test set: Accuracy: 0.768116
Starting BertForSequenceClassification


INFO:__main__:Saving tuned model.
End of defining BertForSequenceClassification
INFO:transformers.configuration_utils:Configuration saved in /opt/ml/model/config.json

INFO:transformers.modeling_utils:Model weights saved in /opt/ml/model/pytorch_model.bin
[2021-08-12 21:20:16.185 algo-1:46 INFO json_config.py:90] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-08-12 21:20:16.186 algo-1:46 INFO hook.py:152] tensorboard_dir has not been set for the hook. SMDebug 


2021-08-12 21:20:52 Uploading - Uploading generated training model
2021-08-12 21:21:50 Completed - Training job completed
Training seconds: 237
Billable seconds: 237


# Host

After training our model, we host it on an Amazon SageMaker Endpoint. To make the endpoint load the model and serve predictions, we implement a few methods in `train_deploy.py`.

* `model_fn()`: function defined to load the saved model and return a model object that can be used for model serving. The SageMaker PyTorch model server loads our model by invoking model_fn.
* `input_fn()`: deserializes and prepares the prediction input. In this example, our request body is first serialized to JSON and then sent to model serving endpoint. Therefore, in `input_fn()`, we first deserialize the JSON-formatted request body and return the input as a `torch.tensor`, as required for BERT.
* `predict_fn()`: performs the prediction and returns the result.

To deploy our endpoint, we call `deploy()` on our PyTorch estimator object, passing in our desired number of instances and instance type:


## Enable Model Monitor Data Capture on the SageMaker Endpoint

In [104]:
from sagemaker.model_monitor import DataCaptureConfig

#s3_capture_upload_path = f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/endpoint/data_capture'
prefix = "sagemaker/CustomModelMonitor"
data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)

print(s3_capture_upload_path)

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path
)

In [105]:
predictor = estimator.deploy(endpoint_name='nlp-data-drift-bert-endpoint',
                             initial_instance_count=1, 
                             instance_type="ml.m4.xlarge",
                             data_capture_config=data_capture_config)

-------------------!

In [111]:
endpoint_name = predictor.endpoint_name
print(endpoint_name)

nlp-data-drift-bert-endpoint


### Predictions using the model

In [107]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [108]:
# batch inference 

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

result = predictor.predict([
    "CLI to download the zip file", 
    "Thanks so much for driving me home",
    "construct the sub-embeddings and corresponding baselines",
    "our Bert model and interpret what the model",
    "Bert models using Captum library",
    "case study we focus on a fine-tuned Question Answering model on SQUAD datase",
    "we pretrain the model, we can load ",
    "need to define baselines / references, nu",
    "defines numericalized special tokens ",
    "Thanks so much for cooking dinner. I really appreciate it",
    "let's define the ground truth for prediction's start and en",
    "pre-computation of embeddings for the second option is necessary because",
    "to summarize attributions for each word token in the sequence.",
    "Nice to meet you, Sergio. So, where are you from"
])

print("Done!")

Sending test traffic to the endpoint nlp-data-drift-bert-endpoint. 
Please wait...
Done!


In [109]:
print("Predicted class: ", np.argmax(result, axis=1))

Predicted class:  [0 1 1 0 1 1 1 0 1 1 0 1 1 1]


### View Captured Data

In [145]:
#Note: It takes a few minutes for the capture data to appear in S3

import boto3

s3_client = boto3.Session().client('s3')

current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/CustomModelMonitor/datacapture/nlp-data-drift-bert-endpoint/AllTraffic/2021/08/12/21/32-28-582-0dafe95a-370b-43d4-85d0-345a952d8ac7.jsonl


In [118]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"[\"CLI to download the zip file\", \"Thanks so much for driving me home\", \"construct the sub-embeddings and corresponding baselines\", \"our Bert model and interpret what the model\", \"Bert models using Captum library\", \"case study we focus on a fine-tuned Question Answering model on SQUAD datase\", \"we pretrain the model, we can load \", \"need to define baselines / references, nu\", \"defines numericalized special tokens \", \"Thanks so much for cooking dinner. I really appreciate it\", \"let's define the ground truth for prediction's start and en\", \"pre-computation of embeddings for the second option is necessary because\", \"to summarize attributions for each word token in the sequence.\", \"Nice to meet you, Sergio. So, where are you from\"]","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[[0.5500634908676147, -0.6241919994

In [119]:
import json

print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "[\"CLI to download the zip file\", \"Thanks so much for driving me home\", \"construct the sub-embeddings and corresponding baselines\", \"our Bert model and interpret what the model\", \"Bert models using Captum library\", \"case study we focus on a fine-tuned Question Answering model on SQUAD datase\", \"we pretrain the model, we can load \", \"need to define baselines / references, nu\", \"defines numericalized special tokens \", \"Thanks so much for cooking dinner. I really appreciate it\", \"let's define the ground truth for prediction's start and en\", \"pre-computation of embeddings for the second option is necessary because\", \"to summarize attributions for each word token in the sequence.\", \"Nice to meet you, Sergio. So, where are you from\"]",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json

### Build an Embedding Matrix from Input Dataset

BERT developers created two main models:

    The BASE: Number of transformer blocks (L): 12, Hidden layer size (H): 768 and Attention heads(A): 12
    The LARGE: Number of transformer blocks (L): 24, Hidden layer size (H): 1024 and Attention heads(A): 16

In this post, I will be using the BASE model as it is smaller and easier to train.

At this point, to make things clearer it is important to understand the special tokens that BERT authors used for fine-tuning and specific task training. These are the following:

    [CLS] : The first token of every sequence. A classification token which is normally used in conjunction with a softmax layer for classification tasks. For anything else, it can be safely ignored.
    [SEP] : A sequence delimiter token which was used at pre-training for sequence-pair tasks (i.e. Next sentence prediction). Must be used when sequence pair tasks are required. When a single sequence is used it is just appended at the end.
    [MASK] : Token used for masked words. Only used for pre-training.
    
The input layer is simply the vector of the sequence tokens along with the special tokens. The “##ing” token in the example above may raise some eyebrows so to clarify, BERT utilizes WordPiece [6] for tokenization which in effect, splits token like “playing” to “play” and “##ing”. This is mainly to cover a wider spectrum of Out-Of-Vocabulary (OOV) words.

Token embeddings are the vocabulary IDs for each of the tokens.

Sentence Embeddings is just a numeric class to distinguish between sentence A and B.

And lastly, Transformer positional embeddings indicate the position of each word in the sequence

In [120]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
sentence_embeddings = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    with torch.no_grad():
        outputs = model(encoded_dict['input_ids'], encoded_dict['attention_mask'])
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        sentence_embeddings.append(sentence_embedding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

In [123]:
sentence_embeddings_list = []

for i in sentence_embeddings:
    sentence_embeddings_list.append(i.numpy())

#### Save the embeddingd as .npy file

In [124]:
np.save('embeddings.npy', sentence_embeddings_list)

In [125]:
sentence_embeddings_list[0]

array([ 7.41458297e-01, -9.18637514e-02,  5.19061983e-01,  4.00024861e-01,
        8.66516009e-02, -3.25883001e-01, -1.24016896e-01,  7.44160190e-02,
        7.17647552e-01, -2.21916839e-01,  1.27307057e-01,  2.71159828e-01,
       -2.26744950e-01,  3.91899735e-01, -1.14975095e-01, -1.54685304e-01,
        5.66386938e-01,  1.60328206e-02,  5.11664748e-02, -8.28863606e-02,
       -2.42579542e-02, -5.17325550e-02, -4.64936376e-01, -4.92869839e-02,
        3.78277659e-01, -2.42931679e-01,  1.66605115e-02,  4.27487679e-02,
       -4.87603545e-01,  3.37469906e-01,  1.70970187e-01, -4.77718234e-01,
       -3.13292176e-01, -5.75484056e-03, -1.34845957e-01,  2.92394936e-01,
       -2.04022348e-01,  4.02555823e-01, -5.67178726e-01,  1.42273039e-01,
       -3.88651818e-01, -3.48770291e-01,  1.54633090e-01, -4.36054543e-02,
       -4.35700864e-01, -2.95430809e-01,  3.90958995e-01,  2.03553393e-01,
       -1.27091154e-01, -5.96861839e-01, -1.63994476e-01,  2.14198381e-01,
       -1.01402458e-02,  

#### Upload the sentence embedding to S3

In [129]:
!aws s3 cp embeddings.npy s3://{bucket}/{model_prefix}/embeddings/

upload: ./embeddings.npy to s3://sagemaker-us-east-1-757967535041/sagemaker/nlp-data-drift-bert-model/embeddings/embeddings.npy


### Evaluation Script

Amazon SageMaker Model Monitor provides a prebuilt container with ability to analyze the data captured from endpoints for tabular datasets. If you would like to bring your own container, Model Monitor provides extension points which you can leverage. 

Under the hood, when you create a MonitoringSchedule, Model Monitor ultimately kicks off processing jobs. Hence the container needs to be aware of the processing job contract 

We need to create an evaluation script that is compatible with container contract inputs and outputs

[Container Contract Inputs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-contract-inputs.html)

[Container Contract Outputs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-contract-outputs.html)

    

In [130]:
!pygmentize docker/evaluation.py

"""Custom Model Monitoring script for Detecting Data Drift in NLP using SageMaker Model Monitor
"""

# Python Built-Ins:
from collections import defaultdict
import datetime
import json
import os
import traceback
from types import SimpleNamespace

# External Dependencies:
import numpy as np
import boto3
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch


def get_environment():
    """Load configuration variables for SM Model Monitoring job

    See https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-contract-inputs.html
    """
    try:
        with open("/opt/ml/config/processingjobconfig.json", "r") as conffile:
            defaults = json.loads(conffile.read())["Environment"]
    except Exception as e:
        traceback.print_exc()
        print("Unable to read environment vars from SM processing config file")
        defaults = {}

    return SimpleNamespace(
        dataset_format=os.environ.get("dataset_format",

### Build and Push Image to ECR

In [131]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'nlp-data-drift-bert-v1'
tag = ':latest'

region = boto3.session.Session().region_name

sm = boto3.client('sagemaker')

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
processing_repository_uri = f'{account_id}.dkr.ecr.{region}.{uri_suffix}/{ecr_repository + tag}'

In [54]:
# Creating the ECR repository and pushing the container image

# SageMaker Classic Notebook Instance:
!docker build -t $ecr_repository docker
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

# SageMaker Studio:
# !cd docker && sm-docker build . --repository $ecr_repository$tag

Sending build context to Docker daemon  17.41kB
Step 1/9 : FROM python:3.7-slim-buster
 ---> b5900b90787e
Step 2/9 : RUN pip3 install sagemaker
 ---> Using cache
 ---> fccd9f2a7991
Step 3/9 : RUN pip3 install scipy
 ---> Using cache
 ---> c247162d8522
Step 4/9 : RUN pip3 install transformers
 ---> Using cache
 ---> c0fa548d853f
Step 5/9 : RUN pip3 install torch
 ---> Using cache
 ---> 5d959178e287
Step 6/9 : RUN pip3 install s3fs
 ---> Using cache
 ---> c31974fc2b7a
Step 7/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 9fa91339885b
Step 8/9 : ADD evaluation.py /
 ---> b64cf3d05d5f
Step 9/9 : ENTRYPOINT ["python3", "/evaluation.py"]
 ---> Running in 567a3862a0bd
Removing intermediate container 567a3862a0bd
 ---> cacf6ab9c602
Successfully built cacf6ab9c602
Successfully tagged nlp-data-drift-bert-v1:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a cre

### Custom Model Monitor for detetcing data drift

In [132]:
from sagemaker.model_monitor import ModelMonitor

monitor = ModelMonitor(
    base_job_name='nlp-data-drift-bert-v1',
    role=role,
    image_uri=processing_repository_uri,
    instance_count=1,
    instance_type='ml.m5.large',
    env={ 'THRESHOLD':'0.5', 'bucket': bucket },
)

In [136]:
from sagemaker.model_monitor import CronExpressionGenerator, MonitoringOutput
from sagemaker.processing import ProcessingInput, ProcessingOutput

destination = f's3://{sagemaker_session.default_bucket()}/{prefix}/{endpoint_name}/monitoring_schedule'

processing_output = ProcessingOutput(
    output_name='result',
    source='/opt/ml/processing/resultdata',
    destination=destination,
)
output = MonitoringOutput(source=processing_output.source, destination=processing_output.destination)

monitor.create_monitoring_schedule(
    monitor_schedule_name='nlp-data-drift-bert-schedule',
    output=output,
    endpoint_input=predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

s3://sagemaker-us-east-1-757967535041/sagemaker/CustomModelMonitor/nlp-data-drift-bert-endpoint/monitoring_schedule


In [137]:
monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:757967535041:monitoring-schedule/nlp-data-drift-bert-schedule',
 'MonitoringScheduleName': 'nlp-data-drift-bert-schedule',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2021, 8, 12, 21, 49, 11, 175000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 12, 21, 49, 15, 594000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinition': {'MonitoringInputs': [{'EndpointInput': {'EndpointName': 'nlp-data-drift-bert-endpoint',
      'LocalPath': '/opt/ml/processing/input/endpoint',
      'S3InputMode': 'File',
      'S3DataDistributionType': 'FullyReplicated'}}],
   'MonitoringOutputConfig': {'MonitoringOutputs': [{'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-757967535041/sagemaker/CustomModelMonitor/nlp-data-drift-bert-endpoint/monitoring_schedule',
       'LocalPath': '/op

In [138]:
jobs = monitor.list_executions()
jobs

No executions found for schedule. monitoring_schedule_name: nlp-data-drift-bert-schedule


[]

In [139]:
if len(jobs) > 0:
    last_execution_desc = monitor.list_executions()[-1].describe()
    print(last_execution_desc)
    print(f'\nExit Message: {last_execution_desc.get("ExitMessage", "None")}')
else:
    print("""No processing job has been executed yet. 
    This means that one hour has not passed yet. 
    You can go to the next code cell and run the processing job manually""")

No processing job has been executed yet. 
    This means that one hour has not passed yet. 
    You can go to the next code cell and run the processing job manually


### Manually execute the processing job

In [143]:
from sagemaker.processing import Processor

processor = Processor(
    base_job_name='nlp-data-drift-bert-v1',
    role=role,
    image_uri=processing_repository_uri,
    instance_count=1,
    instance_type='ml.m5.large',
    env={ 'THRESHOLD':'0.5','bucket': bucket },
)
    
processor.run(
    [ProcessingInput(
        input_name='endpointdata',
        source = "s3://{}/{}/{}".format(bucket, data_capture_prefix,endpoint_name),
        #source=f's3://{sagemaker_session.default_bucket()}/{s3_prefix}/endpoint/data_capture',
        destination = '/opt/ml/processing/input/endpoint',
    )],
    [ProcessingOutput(
        output_name='result',
        source='/opt/ml/processing/resultdata',
        destination=destination,
    )],
)


Job Name:  nlp-data-drift-bert-v1-2021-08-12-21-51-29-927
Inputs:  [{'InputName': 'endpointdata', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-757967535041/sagemaker/CustomModelMonitor/datacapture/nlp-data-drift-bert-endpoint', 'LocalPath': '/opt/ml/processing/input/endpoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-757967535041/sagemaker/CustomModelMonitor/nlp-data-drift-bert-endpoint/monitoring_schedule', 'LocalPath': '/opt/ml/processing/resultdata', 'S3UploadMode': 'EndOfJob'}}]
.............................Starting evaluation with config
namespace(bucket='sagemaker-us-east-1-757967535041', dataset_format=None, dataset_source='/opt/ml/processing/input/endpoint', end_time=None, max_ratio_threshold=0.5, output_path='/opt/ml/processing/resultdata', publish_cloudwatch_metrics=

# Cleanup

Lastly, please remember to delete the monitoring schedule and Amazon SageMaker endpoint to avoid charges:

In [150]:
#Delete the monitoring schedule
monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: nlp-data-drift-bert-schedule


In [151]:
#Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'f82487a1-a635-44ef-a895-3ed7df1f4a20',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f82487a1-a635-44ef-a895-3ed7df1f4a20',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 12 Aug 2021 21:59:19 GMT'},
  'RetryAttempts': 0}}